# Data Analysis Simple Agent

In [13]:
pip install tabulate

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [9]:
import os
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain.agents import AgentType
from langchain_google_genai import ChatGoogleGenerativeAI
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Load environment variables
from dotenv import load_dotenv
import os

# Load environment variables and set OpenAI API key
load_dotenv()
os.environ["GOOGLE_API_KEY"] = os.getenv('GOOGLE_API_KEY')

# Set a random seed for reproducibility
np.random.seed(42)

/Users/tetro/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Lets generate some sample data

In [10]:
n_rows=1000

# Dates
start_date = datetime(2022, 1, 1)
dates = [start_date + timedelta(days=i) for i in range(n_rows)]

# Categories
makes = ['Toyota', 'Honda', 'Ford', 'Chevrolet', 'Nissan', 'BMW', 'Mercedes', 'Audi', 'Hyundai', 'Kia']
models = ['Sedan', 'SUV', 'Truck', 'Hatchback', 'Coupe', 'Van']
colors = ['Red', 'Blue', 'Black', 'White', 'Silver', 'Gray', 'Green']

# Dataset
data = {
    'Date': dates,
    'Make': np.random.choice(makes, n_rows),
    'Model': np.random.choice(models, n_rows),
    'Color': np.random.choice(colors, n_rows),
    'Year': np.random.randint(2015, 2023, n_rows),
    'Price': np.random.uniform(20000, 80000, n_rows).round(2),
    'Mileage': np.random.uniform(0, 100000, n_rows).round(0),
    'EngineSize': np.random.choice([1.6, 2.0, 2.5, 3.0, 3.5, 4.0], n_rows),
    'FuelEfficiency': np.random.uniform(20, 40, n_rows).round(1),
    'SalesPerson': np.random.choice(['Alice', 'Bob', 'Charlie', 'David', 'Eva'], n_rows)
}

# Create DataFrame and sort by date
df = pd.DataFrame(data).sort_values('Date')

In [11]:
df.info(), df.describe(), df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Date            1000 non-null   datetime64[ns]
 1   Make            1000 non-null   object        
 2   Model           1000 non-null   object        
 3   Color           1000 non-null   object        
 4   Year            1000 non-null   int64         
 5   Price           1000 non-null   float64       
 6   Mileage         1000 non-null   float64       
 7   EngineSize      1000 non-null   float64       
 8   FuelEfficiency  1000 non-null   float64       
 9   SalesPerson     1000 non-null   object        
dtypes: datetime64[ns](1), float64(4), int64(1), object(4)
memory usage: 78.2+ KB


(None,
                       Date         Year         Price       Mileage  \
 count                 1000  1000.000000   1000.000000   1000.000000   
 mean   2023-05-15 12:00:00  2018.445000  51145.360800  48484.643000   
 min    2022-01-01 00:00:00  2015.000000  20026.570000     19.000000   
 25%    2022-09-07 18:00:00  2017.000000  36859.940000  23191.500000   
 50%    2023-05-15 12:00:00  2018.000000  52215.155000  47506.000000   
 75%    2024-01-20 06:00:00  2020.000000  65741.147500  73880.250000   
 max    2024-09-26 00:00:00  2022.000000  79972.640000  99762.000000   
 std                    NaN     2.256117  17041.610861  29103.404593   
 
         EngineSize  FuelEfficiency  
 count  1000.000000     1000.000000  
 mean      2.744500       29.688500  
 min       1.600000       20.000000  
 25%       2.000000       24.500000  
 50%       2.500000       29.700000  
 75%       3.500000       34.700000  
 max       4.000000       40.000000  
 std       0.839389        5.896316  ,


## Create Agent

In [29]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

agent = create_pandas_dataframe_agent(
    llm,
    df,
    verbose=True,
    allow_dangerous_code=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

### Define Conversation function

In [30]:
def ask_agent(question):
    """Function to ask questions to the agent and display the response"""
    response = agent.run({
        "input": question,
        "agent_scratchpad": f"Human: {question}\nAI: To answer this question, I need to use Python to analyze the dataframe. I'll use the python_repl_ast tool.\n\nAction: python_repl_ast\nAction Input: ",
    })
    print(f"Question: {question}")
    print(f"Answer: {response}")
    print("---")

### Use

In [35]:
ask_agent("What are the column names in this dataset")
ask_agent("What are the column that are of int type")
ask_agent("What is the most common model")
ask_agent(f"What is the value of top 10% price")

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..




> Entering new AgentExecutor chain...


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..


Thought: I can use the .columns attribute to get the column names of the dataframe.
Action: python_repl_ast
Action Input: print(df.columns)Index(['Date', 'Make', 'Model', 'Color', 'Year', 'Price', 'Mileage',
       'EngineSize', 'FuelEfficiency', 'SalesPerson'],
      dtype='object')


ResourceExhausted: 429 Resource has been exhausted (e.g. check quota).